In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Activation, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Lambda, Input, Concatenate, Multiply, Subtract
from keras.regularizers import l2
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
num_classes = 4
img_size = 250
img_size_final = 50

In [10]:
def load_data():
    
    X = np.load('Lekhan/X_train.npy')
    Y = np.load('Lekhan/Y_train.npy')
    x = np.load('Lekhan/X_test.npy')
    y = np.load('Lekhan/Y_test.npy')
    X = X.reshape(X.shape[0], img_size, img_size, 1).astype('float32')
    x = x.reshape(x.shape[0], img_size, img_size, 1).astype('float32')
    X_mean = X.mean()
    X = (X - X_mean) / 127
    x = (x - X_mean) / 127
    Y = np_utils.to_categorical(Y, num_classes)
    y = np_utils.to_categorical(y, num_classes)
    X = X[:, :img_size_final, :img_size_final, :]
    x = x[:, :img_size_final, :img_size_final, :]
    return (X, Y), (x, y)

In [4]:
def make_pairs(X, Y):

    rx = []
    ry = []
    p = [[], [], [], []]
    for c in range(num_classes):
        for i in range(X.shape[0]):
            if Y[i][c] == 1:
                p[c].append(X[i])
        a = np.array(p[c] + p[c])
        b = a[np.random.permutation(range(a.shape[0]))]
        for i in range(a.shape[0]):
            rx.append([a[i], b[i]])
            ry.append([0, 1])
    sz = (2 * len(ry)) // (num_classes * (num_classes - 1))
    for i in range(num_classes):
        for j in range(i):
            a = np.array(p[i] + p[i])
            a = a[np.random.permutation(range(a.shape[0]))]
            b = np.array(p[j] + p[j])
            b = b[np.random.permutation(range(b.shape[0]))]
            for k in range(sz):
                rx.append([a[k], b[k]])
                ry.append([1, 0])
    return np.array(rx), np.array(ry)

In [5]:
def split_input(inputs):

    output1 = Lambda(lambda x : x[:, 0, :, :, :])(inputs)
    output2 = Lambda(lambda x : x[:, 1, :, :, :])(inputs)
    return output1, output2

In [16]:
conv1_1 = Conv2D(64, (3, 3), activation='relu', padding='same')
conv1_2 = Conv2D(64, (3, 3), activation='relu', padding='same')
pool1 = MaxPooling2D(pool_size=(2, 2))
conv2_1 = Conv2D(128, (3, 3), activation='relu', padding='same')
conv2_2 = Conv2D(128, (3, 3), activation='relu', padding='same')
pool2 = MaxPooling2D(pool_size=(2, 2))
drop = Dropout(0.25)
flat = Flatten()
dens = Dense(256, activation='relu')

def conv_siamese_net(inputs):

    x = conv1_1(inputs)
    x = conv1_2(x)
    x = pool1(x)
    x = conv2_1(x)
    x = conv2_2(x)
    x = pool2(x)
    x = flat(x)
    outputs = dens(x)
    return outputs

In [17]:
def concat(input1, input2):

    x1 = Concatenate()([input1, input2])
    t1 = Subtract()([input1, input2])
    t1 = Multiply()([t1, t1])
    t2 = Multiply()([input1, input2])
    x2 = Concatenate()([t1, t2])
    outputs = Concatenate()([x1, x2])
    return outputs

In [18]:
def fully_connected(inputs):

    x = Dropout(0.5)(inputs)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax')(x)
    return outputs

In [19]:
(X, Y), (x, y) = load_data()
Px, Py = make_pairs(X, Y)
px, py = make_pairs(x, y)

print(Px.shape)
print(Py.shape)
print(px.shape)
print(py.shape)

input_shape = (2, img_size_final, img_size_final, 1)

(2400, 2, 50, 50, 1)
(2400, 2)
(796, 2, 50, 50, 1)
(796, 2)


In [20]:
inputs = Input(shape=input_shape)
x1, x2 = split_input(inputs)
x1 = conv_siamese_net(x1)
x2 = conv_siamese_net(x2)
x = concat(x1, x2)
outputs = fully_connected(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 2, 50, 50, 1)  0                                            
____________________________________________________________________________________________________
lambda_3 (Lambda)                (None, 50, 50, 1)     0           input_2[0][0]                    
____________________________________________________________________________________________________
lambda_4 (Lambda)                (None, 50, 50, 1)     0           input_2[0][0]                    
____________________________________________________________________________________________________
conv2d_5 (Conv2D)                (None, 50, 50, 64)    640         lambda_3[0][0]                   
                                                                   lambda_4[0][0]          

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(Px, Py, batch_size=20, epochs=10, verbose=1, validation_data=(px, py))

Train on 2400 samples, validate on 796 samples
Epoch 1/10
2400/2400 [==============================] - 38s - loss: 0.6317 - acc: 0.6508 - val_loss: 0.5520 - val_acc: 0.7023
Epoch 2/10
2400/2400 [==============================] - 37s - loss: 0.5259 - acc: 0.7346 - val_loss: 0.5517 - val_acc: 0.7186
Epoch 3/10
2400/2400 [==============================] - 37s - loss: 0.4608 - acc: 0.7788 - val_loss: 0.4996 - val_acc: 0.7789
Epoch 4/10
2400/2400 [==============================] - 37s - loss: 0.3759 - acc: 0.8300 - val_loss: 0.5071 - val_acc: 0.7789
Epoch 5/10
2400/2400 [==============================] - 37s - loss: 0.3142 - acc: 0.8750 - val_loss: 0.5341 - val_acc: 0.7889
Epoch 6/10
2400/2400 [==============================] - 37s - loss: 0.2619 - acc: 0.9000 - val_loss: 0.5578 - val_acc: 0.7977
Epoch 7/10
2400/2400 [==============================] - 37s - loss: 0.2141 - acc: 0.9217 - val_loss: 0.4912 - val_acc: 0.8417
Epoch 8/10
2400/2400 [==============================] - 37s - loss: 0.1

In [23]:
score = model.evaluate(px, py, verbose=0)
print('Validation Loss : ', score[0])
print('Validation Accuracy : ', score[1])

Validation Loss :  0.547183154815
Validation Accuracy :  0.86055276352
